In [24]:
# import necessary libraries
import numpy as np
import pandas as pd
import nltk #natural language toolkit
# nltk.download('all')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm
import smtplib
# from sklearn import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
stemmer = nltk.SnowballStemmer("english")
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders


In [25]:
smtp = smtplib.SMTP('smtp.gmail.com', 587)
smtp.ehlo()
smtp.starttls()

(220, b'2.0.0 Ready to start TLS')

In [26]:
smtp.login('khushi.nanwani18@gmail.com', 'ddsu omcu kejn cuhg')

(235, b'2.7.0 Accepted')

In [27]:
df = pd.read_csv('labeled_data.csv')
df.head()

,id,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [30]:
df['labels'] = df['class'].map({0: "Hate Speech Detected", 1: "Offensive Language Detected", 2 : "No Hate and Offensive Speech Detected"})
df.head()

,id,count,hate_speech,offensive_language,neither,class,tweet,labels
0,0,3,0,0,3,2,rt mayasolov woman shouldnt complain clean ho...,No Hate and Offensive Speech Detected
1,1,3,0,3,0,1,rt mlee boy dat coldtyga dwn bad cuffin dat h...,Offensive Language Detected
2,2,3,0,3,0,1,rt urkindofbrand dawg rt babif ever fuck bitc...,Offensive Language Detected
3,3,3,0,2,1,1,rt c_g_anderson viva_bas look like tranni,Offensive Language Detected
4,4,6,0,6,0,1,rt shenikarobert shit hear might true might f...,Offensive Language Detected


In [31]:
df = df[["tweet", "labels"]]

In [32]:
df.head()

,tweet,labels
0,rt mayasolov woman shouldnt complain clean ho...,No Hate and Offensive Speech Detected
1,rt mlee boy dat coldtyga dwn bad cuffin dat h...,Offensive Language Detected
2,rt urkindofbrand dawg rt babif ever fuck bitc...,Offensive Language Detected
3,rt c_g_anderson viva_bas look like tranni,Offensive Language Detected
4,rt shenikarobert shit hear might true might f...,Offensive Language Detected


In [33]:
# preprocessing text
import re
import string
stop_words = set(stopwords.words('english'))
def clean (text):
 text = str(text).lower()
 text = re.sub('\[.*?\]', '', text)
 text = re.sub('https?://\S+|www.\S+', '', text)
 text = re.sub('<.*?>+', '', text)
 text = re.sub(r'[^\w\s]','',text)
 text = re.sub('\n', '', text)
 text = re.sub('\w\d\w', '', text)
 text = [word for word in text.split(' ') if word not in stop_words]
 text=" ".join(text)
 text = [stemmer. stem(word) for word in text. split(' ')]
 text=" ".join(text)
 return text
df["tweet"] = df["tweet"].apply(clean)


<>:7: SyntaxWarning: invalid escape sequence '\['
<>:8: SyntaxWarning: invalid escape sequence '\S'
<>:12: SyntaxWarning: invalid escape sequence '\w'
<>:7: SyntaxWarning: invalid escape sequence '\['
<>:8: SyntaxWarning: invalid escape sequence '\S'
<>:12: SyntaxWarning: invalid escape sequence '\w'
C:\Users\user\AppData\Local\Temp\ipykernel_6068\1555169711.py:7: SyntaxWarning: invalid escape sequence '\['
  text = re.sub('\[.*?\]', '', text)
C:\Users\user\AppData\Local\Temp\ipykernel_6068\1555169711.py:8: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('https?://\S+|www.\S+', '', text)
C:\Users\user\AppData\Local\Temp\ipykernel_6068\1555169711.py:12: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('\w\d\w', '', text)
C:\Users\user\AppData\Local\Temp\ipykernel_6068\1555169711.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [34]:
df.tail()

,tweet,labels
24778,yous muthafin lie 0lifeask pearl corey_emanuel...,Offensive Language Detected
24779,youv gone broke wrong heart babi drove redneck...,No Hate and Offensive Speech Detected
24780,young buck wanna eat dat nigguh like aint fuck...,Offensive Language Detected
24781,youu got wild bitch tellin lie,Offensive Language Detected
24782,ruffl ntac eileen dahlia beauti color combin...,No Hate and Offensive Speech Detected


In [35]:
df.head()

,tweet,labels
0,rt mayasolov woman shouldnt complain clean ho...,No Hate and Offensive Speech Detected
1,rt mlee boy dat coldtyga dwn bad cuffin dat h...,Offensive Language Detected
2,rt urkindofbrand dawg rt babif ever fuck bitc...,Offensive Language Detected
3,rt c_g_anderson viva_ba look like tranni,Offensive Language Detected
4,rt shenikarobert shit hear might true might f...,Offensive Language Detected


In [36]:
# y = mX + c
x=np.array(df["tweet"]) 
y=np.array(df["labels"])

In [37]:

# CountVectorizer is a text preprocessing technique commonly used in natural language processing (NLP) tasks 
# for converting a collection of text documents into a numerical representation
cv = CountVectorizer()

In [38]:

x = cv.fit_transform(x)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size= 0.20, random_state=42 )

In [40]:
#?
model = svm.SVC(kernel='linear', C=1.0)

In [41]:
model.fit(X_train, y_train)

SVC(kernel='linear')

In [15]:
y_pred = model.predict(X_test)

In [16]:
#check Accuracy of The model
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.8892475287472261


In [17]:
# i = "I feel absolutely terrible after hearing the news."
# i = "The loss of my pet dog has absolutely left me heartbroken."
i = "He is a bastard"

df1 = cv.transform([i]).toarray()
print(model.predict(df1))

['Offensive Language Detected']


In [42]:
def send_email(body):
    sender_email = "khushi.nanwani18@gmail.com"  # Replace with your email
    receiver_email = "bhaktinirbhaybhatt@gmail.com"  # Replace with recipient email
    password = "ddsu omcu kejn cuhg"  # Replace with your email password

In [43]:
message = MIMEMultipart()
message["From"] = "khushi.nanwani18@gmail.com"
message["To"] = "bhaktinirbhaybhatt@gmail.com"
message["Subject"] = "Offensive Content Detected"

In [44]:
message.attach(MIMEText("This is an demo email", "plain"))

In [46]:
with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login("khushi.nanwani18@gmail.com", "ddsu omcu kejn cuhg")

In [47]:
server.sendmail("khushi.nanwani18@gmail.com", "bhaktinirbhaybhatt@gmail.com", message.as_string())

SMTPServerDisconnected: please run connect() first